In [61]:
class InventoryAllocator:
    def __init__(self, order, warehouseToInventory):
        assert order != None, "Missing order"
        assert warehouseToInventory != None, "Missing inventory distribution"
        self.order = order
        self.warehouseToInventory = warehouseToInventory
        self.cheapestShipment = list() #list of mappings from warehouse to dictionaries of items to counts
    def getOrder(self):
        return self.order
    def getInventoryDistribution(self):
        return self.warehouseToInventory
    def completeOrder(self):
        for warehouse in self.warehouseToInventory:
            wareToOutput = dict()
            itemsToShipFromThisWarehouse = dict()
            for item in self.order.keys():
                #check if warehouse inventory field is null maybe?
                if item in warehouse["inventory"].keys() and self.order[item] > 0 and warehouse["inventory"][item] > 0:
                    amtToOrder = self.order[item]
                    amtInStock = warehouse["inventory"][item]
                    if amtToOrder <= amtInStock:
                        itemsToShipFromThisWarehouse[item] = amtToOrder
                        warehouse["inventory"][item] -= amtToOrder
                        self.order[item] -= amtToOrder
                    else:
                        itemsToShipFromThisWarehouse[item] = amtInStock
                        warehouse["inventory"][item] = 0
                        self.order[item] -= amtInStock
            if len(itemsToShipFromThisWarehouse) > 0:
                wareToOutput[warehouse["name"]] = itemsToShipFromThisWarehouse
                self.cheapestShipment.append(wareToOutput)
        return self.cheapestShipment

In [55]:
# Example from spec
order = {"apple": 5, "banana": 5, "orange": 5}
invDist = [ { "name": "owd", "inventory": { "apple": 5, "orange": 10 } }, { "name": "dm", "inventory": { "banana": 5, "orange": 10 } } ]
i = InventoryAllocator(order, invDist)
shipment = i.completeOrder()
shipment

[{'owd': {'apple': 5, 'orange': 5}}, {'dm': {'banana': 5}}]

In [56]:
# Exact inventory match
input1 = { "apple": 1 }
inventory1 = [{ "name": "owd", "inventory": { "apple": 1 } }]
i = InventoryAllocator(input1, inventory1)
shipment = i.completeOrder()
shipment

[{'owd': {'apple': 1}}]

In [58]:
# Not enough inventory - no allocations
input2 = { "apple": 1 }
inventory2 = [{ "name": "owd", "inventory": { "apple": 0 } }]
i = InventoryAllocator(input1, inventory1)
shipment = i.completeOrder()
shipment

[]

In [59]:
# Should split an item across warehouses if that's the only way to completely ship an item
input2 = { "apple": 10 }
inventory2 = [{ "name": "owd", "inventory": { "apple": 5 } }, { "name": "dm", "inventory": { "apple": 5 } }]
i = InventoryAllocator(input2, inventory2)
shipment = i.completeOrder()
shipment

[{'owd': {'apple': 5}}, {'dm': {'apple': 5}}]